In [1]:
!pip uninstall -y torch torchvision torchaudio
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install -q diffusers transformers accelerate xformers==0.0.20 gradio plotly
!pip install -q matplotlib seaborn

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 2.0.1+cu118 which is incompatible.


**Imports and Environment Setup**

In [2]:
import os
import torch
import random
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from transformers import pipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import nltk
import gradio as gr

# Download NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)

True

**Environment and Model Initialization**

In [3]:
def setup_environment():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}, CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    return device

def initialize_models(device):
    try:
        # Initializes sentiment analyzer, topic classifier, and image generation model
        sentiment_analyzer = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
        topic_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        
        model_id = "Lykon/dreamshaper-8"
        model = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if str(device) == 'cuda' else torch.float32,
            safety_checker=None
        )
        model.scheduler = DPMSolverMultistepScheduler.from_config(
            model.scheduler.config,
            algorithm_type="dpmsolver++",
            solver_order=2
        )

        if str(device) == 'cuda':
            model.enable_xformers_memory_efficient_attention()
            model.enable_attention_slicing()
            
        model = model.to(device)
        print("Models Initialized Successfully")
        return sentiment_analyzer, topic_classifier, model
    except Exception as e:
        print(f"Error Initializing Models: {e}")
        return None, None, None

**Visualization Functions**

In [4]:
def create_sentiment_pie_chart(sentiment_score, sentiment_label):
    """Create a pie chart visualizing sentiment"""
    labels = ['Positive', 'Negative'] if sentiment_label == 'POSITIVE' else ['Negative', 'Positive']
    values = [sentiment_score, 1 - sentiment_score]
    
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
    fig.update_traces(
        hoverinfo='label+percent', 
        textinfo='percent', 
        textfont_size=15,
        marker=dict(colors=['green', 'red'] if sentiment_label == 'POSITIVE' else ['red', 'green'])
    )
    return fig

def create_topic_bar_chart(topic_scores, topic_labels):
    """Create a bar chart visualizing topic classification"""
    fig = px.bar(
        x=topic_labels, 
        y=topic_scores, 
        labels={'x':'Topics', 'y':'Confidence Score'},
        title='Topic Classification Confidence'
    )
    return fig

**Prompt Analysis Function**

In [5]:
def analyze_prompt(prompt, sentiment_analyzer, topic_classifier):
    # Analyze sentiment of the prompt
    sentiment_result = sentiment_analyzer(prompt)[0]
    
    # Classify potential topics
    candidate_topics = [
        "landscape", "portrait", "abstract", "nature", "urban",
        "fantasy", "sci-fi", "realistic", "artistic", "architectural"
    ]
    topic_result = topic_classifier(prompt, candidate_topics)

    # Determine image style based on keywords
    style_keywords = {
        'realistic': ['realistic', 'photographic', 'natural', 'real'],
        'artistic': ['artistic', 'painted', 'stylized', 'abstract'],
        'fantasy': ['magical', 'fantasy', 'mythical', 'mystical'],
        'sci-fi': ['futuristic', 'sci-fi', 'technological', 'cyber']
    }

    # Calculate style scores based on keyword matches
    style_scores = {style: sum(1 for keyword in keywords if keyword in prompt.lower())
                    for style, keywords in style_keywords.items()}
    primary_style = max(style_scores.items(), key=lambda x: x[1])[0] if any(style_scores.values()) else 'realistic'

    return {
        'sentiment': sentiment_result['label'],
        'sentiment_score': sentiment_result['score'],
        'primary_topic': topic_result['labels'][0],
        'topic_scores': topic_result['scores'],
        'topic_labels': topic_result['labels'],
        'style': primary_style,
    }

**Prompt Enhancement Function**

In [6]:
def generate_enhanced_prompt(original_prompt, analysis):
    # Quality and style enhancement modifiers
    quality_modifiers = [
        "highly detailed",
        "sharp focus",
        "professional photography",
        "8k uhd",
        "golden ratio composition",
        "best quality",
        "masterpiece"
    ]
    
    style_modifiers = {
        'realistic': [
            "photorealistic",
            "hyperrealistic",
            "photograph",
            "documentary style",
            "natural lighting"
        ],
        'artistic': [
            "digital art",
            "concept art",
            "trending on artstation",
            "professional artwork"
        ],
        'fantasy': [
            "ethereal",
            "mystical atmosphere",
            "fantasy art",
            "magical environment"
        ],
        'sci-fi': [
            "futuristic",
            "cinematic lighting",
            "sci-fi atmosphere",
            "technological"
        ]
    }

    # Select style-specific modifiers
    chosen_style_modifiers = style_modifiers.get(analysis['style'], ["photorealistic"])
    subject_emphasis = (
        f"a clear detailed view of {original_prompt}, "
        f"subject centered and in focus, "
        f"main subject prominently featured, "
        f"perfect composition"
    )
    
    # Combine modifiers to create an enhanced prompt
    enhanced_prompt = (
        f"{subject_emphasis}, "
        f"{random.choice(chosen_style_modifiers)}, "
        f"{random.choice(quality_modifiers)}, "
        f"professional lighting, intricate details"
    )
    
    return enhanced_prompt

**Image Generation Function**

In [7]:
def generate_images(model, prompt, device, num_images=3, height=512, width=512, guidance_scale=7.5):
    # Negative prompt to avoid undesired image characteristics
    negative_prompt = (
        "blurry, low quality, low resolution, watermark, text, logo, signature, "
        "deformed, disfigured, mutation, extra limbs, wrong composition, "
        "bad anatomy, wrong anatomy, duplicate, multiple, cross-eye, "
        "out of frame, ugly, extra digit, fewer digits, cropped, jpeg artifacts, "
        "worst quality, poorly drawn, distorted, writing, letters"
    )
    
    images = []
    
    for i in range(num_images):
        # Set random seed for reproducibility
        seed = random.randint(0, 1_000_000)
        generator = torch.Generator(device).manual_seed(seed)
        
        try:
            # Generate image using Stable Diffusion
            image = model(
                prompt,
                height=height,
                width=width,
                num_inference_steps=30,
                guidance_scale=guidance_scale,
                negative_prompt=negative_prompt,
                generator=generator
            ).images[0]
            images.append(image)
        except Exception as e:
            print(f"Error generating image {i+1}: {e}")
            continue
            
    return images

**Main Processing Function**

In [8]:
def process_and_generate(prompt, num_images=3, resolution=512, guidance_scale=7.5):
    # Check if models are initialized
    if not all([SENTIMENT_ANALYZER, TOPIC_CLASSIFIER, MODEL]):
        return None, None, None, "Error: Models not initialized."
    
    try:
        # Clean and validate prompt
        cleaned_prompt = prompt.strip()
        if not cleaned_prompt:
            return None, None, None, "Error: Please enter a prompt."
        
        # Analyze and enhance prompt
        analysis = analyze_prompt(cleaned_prompt, SENTIMENT_ANALYZER, TOPIC_CLASSIFIER)
        enhanced_prompt = generate_enhanced_prompt(cleaned_prompt, analysis)
        
        # Generate images
        images = generate_images(
            MODEL, 
            enhanced_prompt, 
            DEVICE, 
            num_images=num_images, 
            height=resolution, 
            width=resolution,
            guidance_scale=guidance_scale
        )
        
        # Handle image generation failure
        if not images:
            return None, None, None, "Error: Failed to generate images. Please try again."
        
        # Create visualization charts
        sentiment_chart = create_sentiment_pie_chart(
            analysis['sentiment_score'], 
            analysis['sentiment']
        )
        
        topic_chart = create_topic_bar_chart(
            topic_scores=analysis['topic_scores'], 
            topic_labels=analysis['topic_labels']
        )
        
        # Prepare detailed information
        details = (
            f"  Original Prompt:   {cleaned_prompt}\n\n"
            f"  Enhanced Prompt:   {enhanced_prompt}\n\n"
            f"  Style:             {analysis['style']}\n\n"
            f"  Primary Topic:     {analysis['primary_topic']}\n\n"
            f"  Sentiment:         {analysis['sentiment']} (Score: {analysis['sentiment_score']:.2f})"
        )
        
        return images, sentiment_chart, topic_chart, details
    
    except Exception as e:
        return None, None, None, f"Error during generation: {str(e)}"

**Gradio Interface Setup**

In [9]:
# Global model initialization
DEVICE = setup_environment()
SENTIMENT_ANALYZER, TOPIC_CLASSIFIER, MODEL = initialize_models(DEVICE)

# Create Gradio interface
interface = gr.Blocks(theme='default')

with interface:
    # Markdown headers and description
    gr.Markdown("# Emotion-Driven Visuals: Generating Art and Phrases from Text")
    gr.Markdown(
        "A Creative AI System for Emotion-Based Image and Phrase Generation.\n\n"
        "Generate high-quality images from your text descriptions.\n"
        "The system will analyze the sentiment of your input and enhance your prompt while maintaining your original subject matter."
    )
    
    # Input components
    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(
                label="Enter your prompt", 
                placeholder="Describe what you want to generate...", 
                lines=3
            )
            
            # Sliders and dropdowns for customization
            with gr.Row():
                num_images_slider = gr.Slider(
                    minimum=1, maximum=5, value=3, step=1, 
                    label="Number of Images"
                )
                resolution_dropdown = gr.Dropdown(
                    [512, 768, 1024], 
                    value=512, 
                    label="Image Resolution"
                )
                guidance_slider = gr.Slider(
                    minimum=1, maximum=15, value=7.5, step=0.5, 
                    label="Creativity/Guidance Scale"
                )
            
            # Generate and clear buttons
            with gr.Row():
                generate_btn = gr.Button("Generate Images", variant="primary")
                clear_btn = gr.Button("Clear Everything", variant="secondary")
    
    # Output components
    with gr.Row():
        gallery = gr.Gallery(label="Generated Images", show_label=True, elem_id="gallery")
        
        with gr.Column():
            sentiment_plot = gr.Plot(label="Sentiment Analysis")
            topic_plot = gr.Plot(label="Topic Classification")
    
    details_box = gr.Textbox(label="Generation Details", lines=6)
    
    # Button click event handlers
    generate_btn.click(
        fn=process_and_generate, 
        inputs=[prompt_input, num_images_slider, resolution_dropdown, guidance_slider],
        outputs=[gallery, sentiment_plot, topic_plot, details_box]
    )
    
    # Clear button functionality
    clear_btn.click(
        fn=lambda: (None, None, None, "", 3, 512, 7.5, ""),
        inputs=None,
        outputs=[
            gallery, 
            sentiment_plot, 
            topic_plot, 
            details_box, 
            num_images_slider, 
            resolution_dropdown, 
            guidance_slider, 
            prompt_input
        ]
    )

# Launch the Gradio interface
interface.launch(share=True)

Device: cuda, CUDA Available: True
GPU: Tesla T4


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Models Initialized Successfully
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://27e72db1ae9d0c98d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
